In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from evoman.environment import Environment
from controller_custom import player_controller
from multiprocessing import Pool
from scipy.stats import ttest_ind, mannwhitneyu
import pandas as pd
import warnings

pygame 2.6.1 (SDL 2.28.4, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Algorithm A

In [3]:
def run_algorithm_a(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group, elitism_count=1):
	"""
	Runs Genetic Algorithm A with the provided environment and parameters.

	Parameters:
	- env: Evoman Environment instance.
	- npopulation: Population size.
	- gens: Number of generations.
	- mutation_rate: Mutation rate.
	- dom_u: Upper bound for gene values.
	- dom_l: Lower bound for gene values.
	- elitism_count: Number of elites to preserve.

	Returns:
	- history_mean: List of mean fitness per generation.
	- history_max: List of max fitness per generation.
	- best_solution: Best solution found.
	"""
	
	n_hidden_neurons = 10
	n_vars = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5
	
	# Run the simulation and return the fitness
	def simulate(x):
		env.player_controller.set(x, env.get_num_sensors())
		f, _, _, _ = env.play(pcont=x)
		return f
	
	# Evaluate the current population
	def evaluate(population):
		return np.array([simulate(individual) for individual in population])

	# Tournament Selection
	def tournament_selection(population, fitness, k=5):
		selected = []
		for _ in range(len(population)):
			contenders = np.random.choice(len(population), k, replace=False)
			winner = contenders[np.argmax(fitness[contenders])]
			selected.append(population[winner])
		return np.array(selected)

	# Uniform Crossover
	def crossover(parent1, parent2):
		mask = np.random.rand(n_vars) < 0.5
		child1 = np.where(mask, parent1, parent2)
		child2 = np.where(mask, parent2, parent1)
		return child1, child2

	# Gaussian Mutation
	def mutate(child):
		for i in range(n_vars):
			if np.random.rand() < mutation_rate:
				child[i] += np.random.normal(0, 0.1)
				child[i] = np.clip(child[i], dom_l, dom_u)
		return child

	# Initialize population
	population = np.random.uniform(dom_l, dom_u, (npopulation, n_vars))
	fitness = evaluate(population)

	# Record fitness over generations
	history_mean = []
	history_max = []

	# Genetic Algorithm Loop
	for generation in range(1, gens + 1):
		# Selection
		selected = tournament_selection(population, fitness)

		# Crossover
		offspring = []
		for i in range(0, npopulation, 2):
			parent1, parent2 = selected[i], selected[i+1]
			child1, child2 = crossover(parent1, parent2)
			offspring.extend([child1, child2])
		offspring = np.array(offspring)[:npopulation]

		# Mutation
		offspring = np.array([mutate(child) for child in offspring])

		# Evaluation
		offspring_fitness = evaluate(offspring)

		# Replacement: Elitism (preserve top elitism_count individuals)
		elite_indices = np.argsort(fitness)[-elitism_count:]  
		elite_individuals = population[elite_indices]  
		elite_fitness = fitness[elite_indices]  
		worst_indices = np.argsort(offspring_fitness)[:elitism_count]  
		offspring[worst_indices] = elite_individuals  
		offspring_fitness[worst_indices] = elite_fitness  # Replace the fitness of the worst with elite fitness

		# Update population and fitness
		population, fitness = offspring, offspring_fitness

		# Record statistics
		history_mean.append(np.mean(fitness))
		history_max.append(np.max(fitness))

		# Logging
		print(f'Generation {generation}: Best Fitness = {history_max[-1]:.4f}, Mean Fitness = {history_mean[-1]:.4f}')

	# Get best solution
	best_idx = np.argmax(fitness)
	best_solution = population[best_idx]

	np.savetxt(os.path.join('results', f'best_solution_A_{enemy_group}.txt'), best_solution)

	return history_mean, history_max, best_solution


# Algorithm B

In [3]:
def run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group):
    """
    Runs Genetic Algorithm B with Crowding for preserving diversity.

    Parameters:
    - env: Evoman Environment instance.
    - npopulation: Population size.
    - gens: Number of generations.
    - mutation_rate: Mutation rate.
    - dom_u: Upper bound for gene values.
    - dom_l: Lower bound for gene values.
    - crowding_factor: Factor to determine the number of individuals to compete.

    Returns:
    - history_mean: List of mean fitness per generation.
    - history_max: List of max fitness per generation.
    - best_solution: Best solution found.
    """
    
    n_hidden_neurons = 10
    n_vars = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5

    # Run the simulation and return the fitness
    def simulate(x):
        env.player_controller.set(x, env.get_num_sensors())
        f, _, _, _ = env.play(pcont=x)
        return f

    # Evaluate the current population
    def evaluate(population):
        return np.array([simulate(individual) for individual in population])

    # Tournament Selection
    def tournament_selection(population, fitness, k=5):
        selected = []
        for _ in range(len(population)):
            contenders = np.random.choice(len(population), k, replace=False)
            winner = contenders[np.argmax(fitness[contenders])]
            selected.append(population[winner])
        return np.array(selected)

    # Uniform Crossover
    def crossover(parent1, parent2):
        mask = np.random.rand(n_vars) < 0.5
        child1 = np.where(mask, parent1, parent2)
        child2 = np.where(mask, parent2, parent1)
        return child1, child2

    # Gaussian Mutation
    def mutate(child):
        for i in range(n_vars):
            if np.random.rand() < mutation_rate:
                child[i] += np.random.normal(0, 0.1)
                child[i] = np.clip(child[i], dom_l, dom_u)
        return child


    # Initialize population
    population = np.random.uniform(dom_l, dom_u, (npopulation, n_vars))
    fitness = evaluate(population)

    # Record fitness over generations
    history_mean = []
    history_max = []
    history_std = []

    # Genetic Algorithm Loop
    for generation in range(1, gens + 1):
        # Selection
        selected = tournament_selection(population, fitness)

        # Crossover 
        for i in range(0, npopulation, 2):
            parent1, parent2 = selected[i], selected[i + 1]
            child1, child2 = crossover(parent1, parent2)
            
            # Mutation
            child1 = mutate(child1)
            child2 = mutate(child2)

            # Calculate distances (Euclidean) between parents and offspring
            d_p1_o1 = np.linalg.norm(parent1 - child1)
            d_p1_o2 = np.linalg.norm(parent1 - child2)
            d_p2_o1 = np.linalg.norm(parent2 - child1)
            d_p2_o2 = np.linalg.norm(parent2 - child2)

            # Calculate fitness for offspring
            fitness_child1 = simulate(child1)
            fitness_child2 = simulate(child2)

            # Crowding
            if d_p1_o1 + d_p2_o2 < d_p1_o2 + d_p2_o1:
                # Pair child1 with parent1 and child2 with parent2
                if fitness_child1 > fitness[i]:
                    population[i] = child1
                if fitness_child2 > fitness[i + 1]:
                    population[i + 1] = child2
            else:
                # Pair child1 with parent2 and child2 with parent1
                if fitness_child1 > fitness[i + 1]:
                    population[i + 1] = child1
                if fitness_child2 > fitness[i]:
                    population[i] = child2 

        # Recalculate fitness for the updated population
        fitness = evaluate(population)

        # Record statistics for the generation
        history_mean.append(np.mean(fitness))
        history_max.append(np.max(fitness))
        history_std.append(np.std(fitness))

        # Logging
        print(f'Generation {generation}: Best Fitness = {history_max[-1]:.4f}, Mean Fitness = {history_mean[-1]:.4f}, Std = {history_std[-1]:.4f}')

    # Get best solution
    best_idx = np.argmax(fitness)
    best_solution = population[best_idx]

    np.savetxt(os.path.join('results', f'best_solution_B_{enemy_group}.txt'), best_solution)

    return history_mean, history_max, best_solution


# Parameters 

In [4]:
# Experiment Parameters

enemy_group_1 = [1, 4, 5, 8]
enemy_group_2 = [2, 3, 6, 7]
enemy_group_3 = [1,2,3,4,5,6,7,8]
num_runs = 3
npopulation = 100
gens = 50 				
mutation_rate = 0.05
dom_u, dom_l = 1, -1
elitism_count_a = 1
elitism_count_b = 2
 

# The number of hidden neurons for the player controller
n_hidden_neurons = 10

# Initialize data storage
results = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Generation': [],
    'Mean Fitness': [],
    'Max Fitness': [],
    'Best Solution': []
}

best_solutions = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Best Solution': []
}

# Run Algorithm A on Enemy Group 1

In [5]:
print('Running Algorithm A for enemy group 3')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA1_EG3'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_3,
						playermode="ai",
						multiplemode="yes",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_a(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_3')

	for gen in range(gens):
		results['Algorithm'].append('A')
		results['Enemy Group'].append(3)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('A')
	best_solutions['Enemy Group'].append(3)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA1_EG3.csv'), index=False)

Running Algorithm A for enemy group 3
Run 1

MESSAGE: Pygame initialized for simulation.
Generation 1: Best Fitness = 25.7395, Mean Fitness = -2.6913
Generation 2: Best Fitness = 25.7395, Mean Fitness = -0.4156
Generation 3: Best Fitness = 25.7395, Mean Fitness = 2.3165
Generation 4: Best Fitness = 25.7521, Mean Fitness = 5.8077
Generation 5: Best Fitness = 25.7521, Mean Fitness = 8.1058
Generation 6: Best Fitness = 31.1074, Mean Fitness = 9.0180
Generation 7: Best Fitness = 31.1074, Mean Fitness = 10.3297
Generation 8: Best Fitness = 34.0138, Mean Fitness = 10.8815
Generation 9: Best Fitness = 34.0138, Mean Fitness = 10.4333
Generation 10: Best Fitness = 34.0138, Mean Fitness = 11.7689
Generation 11: Best Fitness = 34.0138, Mean Fitness = 14.5952
Generation 12: Best Fitness = 34.0138, Mean Fitness = 15.0628
Generation 13: Best Fitness = 34.0138, Mean Fitness = 17.6837
Generation 14: Best Fitness = 34.0138, Mean Fitness = 18.0578
Generation 15: Best Fitness = 35.3478, Mean Fitness = 20

# Run Algorithm A on Enemy Group 2

In [ ]:
print('Running Algorithm A for enemy group 2')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA1_EG2'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_2,
						playermode="ai",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_a(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_2')

	for gen in range(gens):
		results['Algorithm'].append('A')
		results['Enemy Group'].append(2)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('A')
	best_solutions['Enemy Group'].append(2)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA1_EG2.csv'), index=False)

# Run Algorithm B on Enemy Group 1

In [8]:
print('Running Algorithm B for enemy group 1')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA2_EG1'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_1,
						playermode="ai",
						multiplemode="yes",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_1')

	for gen in range(gens):
		results['Algorithm'].append('B')
		results['Enemy Group'].append(1)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('B')
	best_solutions['Enemy Group'].append(1)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA2_EG1.csv'), index=False)

Running Algorithm B for enemy group 1
Run 1

MESSAGE: Pygame initialized for simulation.
Generation 1: Best Fitness = 30.4205, Mean Fitness = -2.1405, Std = 7.1852
Generation 2: Best Fitness = 30.4205, Mean Fitness = 2.4706, Std = 8.8202
Generation 3: Best Fitness = 54.5532, Mean Fitness = 8.5504, Std = 10.8260
Generation 4: Best Fitness = 54.5532, Mean Fitness = 12.1882, Std = 11.4531
Generation 5: Best Fitness = 54.5532, Mean Fitness = 14.2524, Std = 10.9986
Generation 6: Best Fitness = 54.5532, Mean Fitness = 16.3811, Std = 10.7962
Generation 7: Best Fitness = 54.5532, Mean Fitness = 18.3991, Std = 10.6394
Generation 8: Best Fitness = 56.8929, Mean Fitness = 20.8868, Std = 11.2305
Generation 9: Best Fitness = 56.8929, Mean Fitness = 23.5312, Std = 12.4095
Generation 10: Best Fitness = 56.8929, Mean Fitness = 25.5719, Std = 12.3482
Generation 11: Best Fitness = 56.8929, Mean Fitness = 27.3301, Std = 12.6113
Generation 12: Best Fitness = 56.8929, Mean Fitness = 28.7807, Std = 12.7220


# Run Algorithm B on Enemy Group 2

In [5]:
print('Running Algorithm B for enemy group 2')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA2_EG2'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_2,
						playermode="ai",
						multiplemode="yes",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_2')

	for gen in range(gens):
		results['Algorithm'].append('B')
		results['Enemy Group'].append(2)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('B')
	best_solutions['Enemy Group'].append(2)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA2_EG2.csv'), index=False)

Running Algorithm B for enemy group 2
Run 1

MESSAGE: Pygame initialized for simulation.
Generation 1: Best Fitness = 50.4251, Mean Fitness = 1.3393, Std = 10.7974
Generation 2: Best Fitness = 50.4251, Mean Fitness = 7.7612, Std = 11.8357
Generation 3: Best Fitness = 50.4251, Mean Fitness = 14.3465, Std = 12.2467
Generation 4: Best Fitness = 56.1075, Mean Fitness = 19.8592, Std = 11.2011
Generation 5: Best Fitness = 56.1075, Mean Fitness = 23.6765, Std = 9.4320
Generation 6: Best Fitness = 56.1075, Mean Fitness = 25.5186, Std = 7.8409
Generation 7: Best Fitness = 61.4108, Mean Fitness = 27.0473, Std = 7.3938
Generation 8: Best Fitness = 61.4108, Mean Fitness = 27.9798, Std = 7.0362
Generation 9: Best Fitness = 61.4108, Mean Fitness = 28.9966, Std = 7.6452
Generation 10: Best Fitness = 61.4108, Mean Fitness = 30.0547, Std = 8.0673
Generation 11: Best Fitness = 61.4108, Mean Fitness = 32.0879, Std = 9.0607
Generation 12: Best Fitness = 61.4108, Mean Fitness = 33.3667, Std = 9.4225
Genera